https://jira.x5.ru/browse/CVMUPG-794

In [1]:
name = 'CVMUPG-794'

In [2]:
import os
import re
from pyspark.sql import SparkSession
from typing import List, Dict, Callable
import socket

spark = None

EXECUTOR_ENV = 'hdfs:///share/products/cvm5/lib/python/anaconda_2.4.4_ds.tar.gz'  # 2.4.4 
SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.4.4.zip'                     # 2.4.4
#EXECUTOR_ENV = 'hdfs:///share/lib/python/env/anaconda-2019.07.tar.gz'
#SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.3.1.zip'

os.environ["ARROW_LIBHDFS_DIR"] = "/usr/hdp/2.6.5.0-292/usr/lib"
os.environ['HADOOP_HOME'] = '/usr/hdp/current/hadoop-client/'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf/'
os.environ['SPARK_HOME'] = '/opt/conda/lib/python3.7/site-packages/pyspark'
os.environ['PYSPARK_PYTHON'] = 'anaconda_2.4.4_ds.tar.gz/bin/python3'             # 2.4.4
#os.environ['PYSPARK_PYTHON'] = 'anaconda-2019.07.tar.gz/bin/python3'






def restart_spark(task_name: str, num_executors: int, executor_memory='4G', executor_cores=2,
                  driver_memory='2G', queue='cvm5-rnd', additional_params: Dict[str, str] = None):
    global spark

    if spark:
        sc = spark.sparkContext
        if sc and sc._jsc:
            if not sc._jsc.sc().isStopped():
                print('Using cached spark')
                return sc, spark

    need_ports_for_app = 3
    user_tcp_ports = _get_user_tcp_ports()
    free_ports = _get_free_ports(user_tcp_ports)
    assert len(free_ports) >= need_ports_for_app, \
        f"Not enough free ports ({len(free_ports)}), need {need_ports_for_app}, stop other apps"
    app_ports = free_ports[:need_ports_for_app]

    host_ip = os.getenv('HOST_IP')
    
    spark_session = (
        SparkSession
        .builder
        .appName(task_name)
        .master('yarn')
        .config('spark.driver.memory', driver_memory)
        .config('spark.driver.maxResultSize', driver_memory)
        .config('spark.executor.cores', executor_cores)
        .config('spark.executor.memory', executor_memory)
        .config('spark.executor.memoryOverhead', '1G')
        .config('spark.dynamicAllocation.enabled', 'true')
        .config('spark.dynamicAllocation.maxExecutors', num_executors)
        .config('spark.sql.broadcastTimeout', '36000')
        .config('spark.dynamicAllocation.cachedExecutorIdleTimeout', '1200s')
        .config('spark.ui.port', app_ports[0])
        .config('spark.blockManager.port', app_ports[1])
        .config('spark.driver.port', app_ports[2])
        .config('spark.driver.host', host_ip)
        .config('spark.driver.bindAddress', '0.0.0.0')
        .config('spark.driver.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
        .config('spark.driver.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.debug.maxToStringFields', '50')
        .config('spark.yarn.queue', queue)
        .config('spark.yarn.dist.archives', EXECUTOR_ENV)
        .config('spark.yarn.archive', SPARK_ARCHIVE)
        .config('spark.yarn.am.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.rpc.message.maxSize', '1024')
        .config('spark.sql.warehouse.dir', '/apps/hive/warehouse')
        .config('spark.sql.execution.pandas.respectSessionTimeZone', 'false')
        .config('spark.sql.orc.filterPushdown', 'true')
        .config('spark.sql.hive.convertMetastoreOrc', 'true')
        .config('spark.shuffle.service.enabled', 'true')
        .config('spark.hadoop.yarn.timeline-service.enabled', 'false')
        .config('spark.hadoop.yarn.client.failover-proxy-provider',
                'org.apache.hadoop.yarn.client.ConfiguredRMFailoverProxyProvider')
        .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
        .config('spark.kryoserializer.buffer.max', '128m')
        .config('spark.executor.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
    )

    if additional_params:
        for key, value in additional_params.items():
            spark_session = spark_session.config(key, value)

    spark = (
        spark_session
        .enableHiveSupport()
        .getOrCreate()
    )
    sc = spark.sparkContext

    return sc, spark


def _get_user_tcp_ports() -> List[str]:
    regexp = re.compile(r'-2e')
    envuser= os.getenv('HOSTNAME')
    if regexp.search(envuser):       
      _, user_name, user_surname = envuser.upper().split('-')
      user_full_name = '_'.join([user_name, user_surname])
    else:
      _, user_name  = envuser.upper().split('-') 
      user_full_name = user_name
    user_tcp_ports = [v for k, v in os.environ.items() if user_full_name in k and k.endswith('TCP_PORT')]
    return user_tcp_ports


def _get_free_ports(ports: List[str]):
    free_ports = []
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('0.0.0.0', int(port))) != 0:
                free_ports.append(port)
    return free_ports


In [3]:
sc, spark = restart_spark(
                    name, 
                    21, 
                    executor_memory='5G', 
                    executor_cores=3, 
                    driver_memory='7G', 
                    additional_params={"spark.sql.shuffle.partitions": "300"})

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/25 14:57:25 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
sc.setLogLevel('ERROR')

In [5]:
import pandas as pd
import sys
from pyspark.sql import functions as F
import datetime
from datetime import timedelta

sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 
from segmentation import *

In [6]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

# seg1

### Выбираем гостей нужного юзкейса

In [7]:
usecase = ['churn', 'frequency', 'cross', 'upgrade', 'ump'] #churn, frequency, cross+upgrade+ump
dt = datetime.date.today()

In [9]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )

In [10]:
customers_usecase.count()

49276270

### По этой активной базе отсекаем ГКГ, проверяем доступность в канале смс вайбер на 13 декабря.

In [8]:
check_date = '2021-12-13'

In [12]:
guests = remove_global_control_group(spark, customers_usecase, check_date)

 Фильтр Глобальная контрольная группа - валиден.


[Stage 10:=====================================================>(295 + 5) / 300]

	 Отсёк 3 466 512 записей.


In [13]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:41


In [14]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:34


In [15]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:17


In [16]:
seg_sms4 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[3], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:37


In [17]:
seg_sms5 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[4], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:15


In [18]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).union(seg_sms4).union(seg_sms5).distinct()

In [19]:
seg_sms.write.parquet('temp11', mode='overwrite')
seg_sms = spark.read.parquet('temp11')

[Stage 81:>(2 + 12) / 108][Stage 82:>  (9 + 4) / 64][Stage 84:>  (2 + 0) / 64]0]800]]]21/11/25 12:57:04 ERROR client.TransportClient: Failed to send RPC RPC 4973686878317076939 to /192.168.234.57:59216: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
21/11/25 12:57:04 ERROR cluster.YarnScheduler: Lost executor 41 on mn-hd0238.x5.ru: Slave lost
[Stage 81:>(3 + 12) / 108][Stage 82:>  (9 + 4) / 64][Stage 84:>  (2 + 0) / 64]21/11/25 12:57:04 ERROR client.TransportClient: Failed to send RPC RPC 6210240559142910480 to /192.168.234.57:59216: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
21/11/25 12:57:04 ERROR cluster.YarnScheduler: Lost executor 55 on mn-hd0217.x5.ru: Slave lost
21/11/25 12:57:05 ERROR client.TransportClient: Failed to send RPC RPC 5142988007691008796

In [20]:
seg_sms.count()

17751644

### Собираем чеки этих гостей

In [9]:
start_date = datetime.date(2021, 8, 15)
end_date = datetime.date(2021, 11, 15)

In [10]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [23]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [76]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [11]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


#### Отбираем тех, кто покупал продукты в категории шоколад

##### Отбираем PLU определенной категории

In [26]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk') == plu_hierarchy_lvl_4_dk)
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
                 .filter(F.col('syntethic_category_id') == 116)
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [27]:
checks_items_choko = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [28]:
checks_tc5_choko = checks_items_choko.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [29]:
seg_choko = (checks_tc5_choko.select('customer_rk')
                  .distinct()
      )

#### Сегмент потребителей шоколада за 3 месяца готов

In [30]:
seg_choko.write.parquet('temp12', mode='overwrite')
seg_choko = spark.read.parquet('temp12')

In [31]:
seg_choko.count()

8125010

In [34]:
seg_choko_churn = seg_choko.join(uc_churn, 'customer_rk', 'inner')
seg_choko_frequency = seg_choko.join(uc_frequency, 'customer_rk', 'inner')
seg_choko_cross = seg_choko.join(uc_cross, 'customer_rk', 'inner')
seg_choko_upgrade = seg_choko.join(uc_upgrade, 'customer_rk', 'inner')
seg_choko_ump = seg_choko.join(uc_ump, 'customer_rk', 'inner')

In [35]:
seg_choko_churn_qty = seg_choko_churn.count()
seg_choko_frequency_qty = seg_choko_frequency.count()
seg_choko_cross_qty = seg_choko_cross.count()
seg_choko_upgrade_qty = seg_choko_upgrade.count()
seg_choko_ump_qty = seg_choko_ump.count()

In [36]:
print('churn: {}, frequency: {}, cross: {}, upgrade: {}, ump: {}'.format(seg_choko_churn_qty, 
                                                                         seg_choko_frequency_qty,
                                                                         seg_choko_cross_qty,
                                                                         seg_choko_upgrade_qty,
                                                                         seg_choko_ump_qty))

churn: 813812, frequency: 3502005, cross: 1444087, upgrade: 1775442, ump: 589664


In [54]:
seg_choko_all = seg_choko_churn_qty + seg_choko_frequency_qty + seg_choko_cross_qty + seg_choko_upgrade_qty + seg_choko_ump_qty

In [55]:
seg_choko_all

8125010

In [56]:
print(round(seg_choko_churn_qty/seg_choko_all*100), round(seg_choko_frequency_qty/seg_choko_all*100), 
      round(seg_choko_cross_qty/seg_choko_all*100), round(seg_choko_upgrade_qty*100/seg_choko_all), 
      round(seg_choko_ump_qty*100/seg_choko_all))

10 43 18 22 7


#### Отбираем тех, кто покупал продукты в категории шоколад бренд Альпенгольд

##### Отбираем PLU определенной категории

In [32]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_choko, on='customer_rk', how='inner')

In [33]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [34]:
plu_codes_cat2 = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk') == plu_hierarchy_lvl_4_dk)
                 .filter(F.col('plu_brand_code') == '0247')
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [35]:
checks_items_choko_alpen = checks_items.filter(F.col('plu_code').isin(plu_codes_cat2)) # только чеки с нужными plu

In [36]:
checks_tc5_choko_alpen = checks_items_choko_alpen.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [37]:
seg_choko_alpen = (checks_tc5_choko_alpen.select('customer_rk')
                  .distinct()
      )

#### Сегмент потребителей шоколада Альпенгольд за 3 месяцев готов

In [38]:
seg_choko_alpen.count()

2722051

##### Смотрим гостей, которые не покупали шоколад - покупал ли кто-то из них печенье?

In [39]:
seg_no_choko = seg_sms.join(seg_choko, 'customer_rk', how='left_anti')

In [40]:
seg_no_choko.count()

9626634

In [41]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_no_choko, on='customer_rk', how='inner')

In [42]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [43]:
plu_codes_cat_cookie = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk') == plu_hierarchy_lvl_4_dk)
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
                 .filter(F.col('plu_hierarchy_lvl_4_dk') == 'FD0607002')
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [44]:
checks_items_cookie = checks_items.filter(F.col('plu_code').isin(plu_codes_cat_cookie)) # только чеки с нужными plu

In [45]:
checks_tc5_cookie = checks_items_cookie.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [46]:
seg_cookie = (checks_tc5_cookie.select('customer_rk')
                  .distinct()
      )

#### Сегмент потребителей печенья, из тех, кто не покупает шоколад

In [47]:
seg_cookie.write.parquet('temp13', mode='overwrite')
seg_cookie = spark.read.parquet('temp13')

In [48]:
seg_cookie.count()

1462209

In [31]:
seg_cookie_churn = seg_cookie.join(uc_churn, 'customer_rk', 'inner')
seg_cookie_frequency = seg_cookie.join(uc_frequency, 'customer_rk', 'inner')
seg_cookie_cross = seg_cookie.join(uc_cross, 'customer_rk', 'inner')
seg_cookie_upgrade = seg_cookie.join(uc_upgrade, 'customer_rk', 'inner')
seg_cookie_ump = seg_cookie.join(uc_ump, 'customer_rk', 'inner')

In [32]:
seg_cookie_churn_qty = seg_cookie_churn.count()
seg_cookie_frequency_qty = seg_cookie_frequency.count()
seg_cookie_cross_qty = seg_cookie_cross.count()
seg_cookie_upgrade_qty = seg_cookie_upgrade.count()
seg_cookie_ump_qty = seg_cookie_ump.count()

In [33]:
print('churn: {}, frequency: {}, cross: {}, upgrade: {}, ump: {}'.format(seg_cookie_churn_qty, 
                                                                         seg_cookie_frequency_qty,
                                                                         seg_cookie_cross_qty,
                                                                         seg_cookie_upgrade_qty,
                                                                         seg_cookie_ump_qty))

churn: 214436, frequency: 819324, cross: 144212, upgrade: 217410, ump: 66827


In [51]:
seg_cookie_all = seg_cookie_churn_qty+seg_cookie_frequency_qty+seg_cookie_cross_qty+seg_cookie_upgrade_qty+seg_cookie_ump_qty

In [52]:
seg_cookie_all

1462209

In [53]:
print(round(seg_cookie_churn_qty/seg_cookie_all*100), round(seg_cookie_frequency_qty/seg_cookie_all*100), 
      round(seg_cookie_cross_qty/seg_cookie_all*100), round(seg_cookie_upgrade_qty*100/seg_cookie_all), 
      round(seg_cookie_ump_qty*100/seg_cookie_all))

15 56 10 15 5


### Из тех, кто не покупал печенье, покупал молочку и кисломолочку?

In [12]:
seg_sms = spark.read.parquet('temp11')
seg_choko = spark.read.parquet('temp12')
seg_no_choko = seg_sms.join(seg_choko, 'customer_rk', how='left_anti')
seg_cookie = spark.read.parquet('temp13')

In [13]:
seg_no_cookie = seg_no_choko.join(seg_cookie, 'customer_rk', how='left_anti')

In [14]:
seg_no_cookie.count()

8164425

In [15]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_no_cookie, on='customer_rk', how='inner')

In [16]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [17]:
plu_codes_cat_milk = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk') == plu_hierarchy_lvl_4_dk)
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
                 .filter(F.col('syntethic_category_id').isin([26, 79, 49, 51, 52, 53, 76, 28, 29, 32]))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [19]:
checks_items_milk = checks_items.filter(F.col('plu_code').isin(plu_codes_cat_milk)) # только чеки с нужными plu

21/11/25 15:01:14 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.57:53110 is closed
21/11/25 15:01:14 ERROR client.TransportClient: Failed to send RPC RPC 5656043214876233424 to /192.168.234.57:53110: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)


In [20]:
checks_tc5_milk = checks_items_milk.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [21]:
seg_milk = (checks_tc5_milk.select('customer_rk')
                  .distinct()
      )

#### Сегмент потребителей молочки и кисломолочки, из тех, кто не покупает печенье

In [22]:
seg_milk.write.parquet('temp14', mode='overwrite')
seg_milk = spark.read.parquet('temp14')

In [23]:
seg_milk.count()

3056435

In [27]:
seg_milk_churn = seg_milk.join(uc_churn, 'customer_rk', 'inner')
seg_milk_frequency = seg_milk.join(uc_frequency, 'customer_rk', 'inner')
seg_milk_cross = seg_milk.join(uc_cross, 'customer_rk', 'inner')
seg_milk_upgrade = seg_milk.join(uc_upgrade, 'customer_rk', 'inner')
seg_milk_ump = seg_milk.join(uc_ump, 'customer_rk', 'inner')

In [40]:
seg_milk_churn_qty = seg_milk_churn.count()
seg_milk_frequency_qty = seg_milk_frequency.count()
seg_milk_cross_qty = seg_milk_cross.count()
seg_milk_upgrade_qty = seg_milk_upgrade.count()
seg_milk_ump_qty = seg_milk_ump.count()

In [41]:
print('churn: {}, frequency: {}, cross: {}, upgrade: {}, ump: {}'.format(seg_milk_churn_qty, 
                                                                         seg_milk_frequency_qty,
                                                                         seg_milk_cross_qty,
                                                                         seg_milk_upgrade_qty,
                                                                         seg_milk_ump_qty))

churn: 697489, frequency: 1884721, cross: 152807, upgrade: 256081, ump: 65337


In [50]:
print(round(seg_milk_churn_qty/seg_milk_all*100), round(seg_milk_frequency_qty/seg_milk_all*100), 
      round(seg_milk_cross_qty/seg_milk_all*100), round(seg_milk_upgrade_qty*100/seg_milk_all), 
      round(seg_milk_ump_qty*100/seg_milk_all))

23 62 5 8 2


In [43]:
seg_milk_all = seg_milk_churn_qty+seg_milk_frequency_qty+seg_milk_cross_qty+seg_milk_upgrade_qty+seg_milk_ump_qty

In [45]:
seg_milk_all

3056435

In [26]:
uc_churn = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase') == 'churn')
                     .select('customer_rk')
                            )

uc_frequency = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase') == 'frequency')
                     .select('customer_rk')
                            )

uc_cross = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase') == 'cross')
                     .select('customer_rk')
                            )

uc_upgrade = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase') == 'upgrade')
                     .select('customer_rk')
                            )

uc_ump = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase') == 'ump')
                     .select('customer_rk')
                            )

## По базе частоты

### Выбираем гостей нужного юзкейса

In [8]:
usecase = ['churn', 'frequency', 'cross', 'upgrade', 'ump'] #churn, frequency, cross+upgrade+ump
dt = datetime.date.today()
lifetime = 90

In [9]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase').isin(usecase))
                     .filter(F.col('lifetime') >= lifetime)
                     .select('account_no', 'customer_rk')
                            )

In [10]:
customers_usecase.count()

46080470

In [ ]:
seg_sms_pd = segm.toPandas()

In [ ]:
len(seg_sms_pd)

In [ ]:
seg_sms_pd.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [ ]:
seg_sms_pd = seg_sms_pd.astype('str')

In [ ]:
cross = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('1')) | (seg_sms_pd['customer_rk'].str.endswith('2'))
              | (seg_sms_pd['customer_rk'].str.endswith('3')) | (seg_sms_pd['customer_rk'].str.endswith('4'))].reset_index(drop=True).copy()

In [ ]:
upgrade = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('5')) | (seg_sms_pd['customer_rk'].str.endswith('6'))
              | (seg_sms_pd['customer_rk'].str.endswith('7')) | (seg_sms_pd['customer_rk'].str.endswith('8'))].reset_index(drop=True).copy()

In [ ]:
ump = seg_sms_pd.loc[(seg_sms_pd['customer_rk'].str.endswith('0'))
              | (seg_sms_pd['customer_rk'].str.endswith('9'))].reset_index(drop=True).copy()

In [ ]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)

In [ ]:
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)

In [ ]:
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [ ]:
print('cross: {}, upgrade: {}, ump: {}'.format(len(cross), len(upgrade), len(ump)))